In [1]:
from pathlib import Path
from aqua_tracekit import SdtModel, SdtSchema, Aggregation
from pathlib import Path
from datetime import datetime
import polars as pl

In this example, we trace forward using the data from example 3.
We trace only the population stocked in C1 forward to study the 

In [ ]:
# use data from example 3. 
base_path = Path("../example_3_mix/data")
model = SdtModel(base_path=str(base_path.resolve()))
# load model
df_containers = model.load_containers("containers.csv")
df_populations = model.load_populations("populations.csv")
df_transfers = model.load_transfers("transfers.csv")
# trace the population starting in C1
df_origin_populations = model.get_populations_incoming()
df_origin_populations = df_origin_populations.filter(pl.col(SdtSchema.Container.CONTAINER_ID)=="C1")

df_traceability_index = model.trace_populations(df_origin_populations)
# add o2 data measured on containers
df_cage_o2 = model.load_container_timeseries("O2.csv")
df_cage_o2 = model.parse_float(df_cage_o2, "O2_mg_per_litre")
df_pop_o2 = model.map_container_data_to_populations(df_cage_o2, include_unmatched=False, allow_multiple=False)
df_traced_data = model.add_data_to_trace(df_pop_o2, df_traceability_index)
# aggregate to min, max and contribution breakdown
aggs = [
    Aggregation.min("O2_mg_per_litre"),
    Aggregation.max("O2_mg_per_litre"),
    Aggregation.contribution_breakdown([SdtSchema.TraceabilityIndex.TRACED_POPULATION_ID, SdtSchema.TraceabilityIndex.FACTORS.SHARE_COUNT_BACKWARD], alias = "part_of_source"),
    Aggregation.contribution_breakdown([SdtSchema.TraceabilityIndex.TRACED_POPULATION_ID, SdtSchema.TraceabilityIndex.FACTORS.SHARE_COUNT_FORWARD], alias = "part_of_target")
]
df_grouped_data = model.aggregate_traced_data(df_traced_data, aggs)
df_grouped_data = df_grouped_data.sort(SdtSchema.TimeSeries.DATE_TIME)

df_grouped_data.head(11)


origin_population_id,date_time,O2_mg_per_litre_min,O2_mg_per_litre_max,part_of_source,part_of_target
str,datetime[μs],f64,f64,str,str
"""P1""",2025-09-01 16:00:00,4.0,4.0,"""P1:1.0""","""P1:1.0"""
"""P1""",2025-09-02 16:00:00,5.0,5.0,"""P1:1.0""","""P1:1.0"""
"""P1""",2025-09-03 16:00:00,4.0,4.0,"""P1:1.0""","""P1:1.0"""
"""P1""",2025-09-04 16:00:00,5.0,5.0,"""P1:1.0""","""P1:1.0"""
"""P1""",2025-09-05 16:00:00,4.0,4.0,"""P3:0.1""","""P3:1.0"""
"""P1""",2025-09-06 16:00:00,5.0,5.0,"""P3:0.1""","""P3:1.0"""
"""P1""",2025-09-07 16:00:00,4.0,4.0,"""P3:0.1""","""P3:1.0"""
"""P1""",2025-09-08 16:00:00,5.0,5.0,"""P3:0.1""","""P3:1.0"""
"""P1""",2025-09-09 16:00:00,4.0,4.0,"""P3:0.1""","""P3:1.0"""
